# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates.
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [ ]:
import cudf
import cuml

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [ ]:
gdf = cudf.read_csv('./data/week3.csv')

## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [ ]:
gdf['infected'] = gdf['infected'].astype('float32')
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [ ]:
gr = gdf[['employment', 'infected']].groupby('employment')
gr.count()

,infected
employment,
K,1122406
X,181988
V,10098466
Z,7161907
"B, D, E",486785
C,2653753
M,2214336
O,1843446
U,12459115


In [ ]:
infected_df = gdf[gdf['infected']==1].reset_index()
grinf = infected_df[['employment', 'infected']].groupby('employment')
grinf.count()

,infected
employment,
"R, S, T",8997
Q,48505
G,17561
V,76648
I,16116
M,10578
F,6604
O,9741
N,6541


In [ ]:
rate = grinf.count()/gr.count() * 100
rate

,infected
employment,
A,0.385276
"B, D, E",0.377374
C,0.388167
F,0.318169
G,0.494751
H,0.338758
I,1.035350
J,0.393944
K,0.477189


In [ ]:
rate.sort_values('infected', ascending = False)

,infected
employment,
Q,1.275574
I,1.035350
V,0.759006
P,0.619031
Z,0.565464
"R, S, T",0.539002
O,0.528413
L,0.497013
G,0.494751


Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [ ]:
code_guide_gdf = cudf.read_csv('./data/code_guide.csv')
code_guide_gdf

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [ ]:
grse = gdf[['employment','sex','infected']].groupby(['sex','employment'], sort = True)
grse.count()

infected
sex employment          
f   A              82229
    B, D, E       115888
    C             688196
    F             266193
    G            1682827
    H             258211
    I             819431
    J             322584
    K             494256
    L             191242
    M             975240
    N             633114
    O             976302
    P            2169494
    Q            2954725
    R, S, T       913091
    U            6073869
    V            5502736
    X              77253
    Z            4382232
m   A             223526
    B, D, E       370897
    C            1965557
    F            1809435
    G            1866638
    H            1140131
    I             737144
    J             857788
    K             628150
    L             155228
    M            1239096
    N             734023
    O             867144
    P             836655
    Q             847877
    R, S, T       756106
    U            6385246
    V            4595730
    X             104735
    Z            2779675

In [ ]:
grseinf = infected_df[['employment','sex','infected']].groupby(['sex','employment'], sort = True)
grseinf.count()

infected
sex employment          
f   A                616
    B, D, E          924
    C               5251
    F               2017
    G              12715
    H               1907
    I              12344
    J               2466
    K               3792
    L               1381
    M               7456
    N               4678
    O               7536
    P              16453
    Q              44164
    R, S, T         7075
    U               2001
    V              59713
    X                571
    Z              33430
m   A                562
    B, D, E          913
    C               5050
    F               4587
    G               4846
    H               2830
    I               3772
    J               2184
    K               1564
    L                341
    M               3122
    N               1863
    O               2205
    P               2156
    Q               4341
    R, S, T         1922
    U                701
    V              16935
    X                255
    Z               7068

In [ ]:
ratese = grseinf.count()/grse.count() * 100
ratese

infected
sex employment          
f   A           0.749127
    B, D, E     0.797322
    C           0.763009
    F           0.757721
    G           0.755574
    H           0.738543
    I           1.506411
    J           0.764452
    K           0.767214
    L           0.722122
    M           0.764530
    N           0.738887
    O           0.771892
    P           0.758380
    Q           1.494691
    R, S, T     0.774841
    U           0.032944
    V           1.085151
    X           0.739130
    Z           0.762853
m   A           0.251425
    B, D, E     0.246160
    C           0.256925
    F           0.253505
    G           0.259611
    H           0.248217
    I           0.511705
    J           0.254608
    K           0.248985
    L           0.219677
    M           0.251958
    N           0.253807
    O           0.254283
    P           0.257693
    Q           0.511985
    R, S, T     0.254197
    U           0.010978
    V           0.368494
    X           0.243472
    Z           0.254274

## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)